# Gell-Mann matrix

Gell-Mann matrices are a set of Hermitian matrices.

In [ ]:
import numpy as np
import numqi

np_rng = np.random.default_rng()


## d=3

Usually, they denotes the following $8$ matrices with $3$ columns and $3$ rows. [wiki/Gell-Mann-matrices](https://en.wikipedia.org/wiki/Gell-Mann_matrices) One can retrieve these matrices matrices via `numqi.gellmann` module as follows. The order of the matrices is different from the one in the wiki page and `numqi` chose this one for easier manipulation when $d$ is not $3$.

In [ ]:
gm3 = numqi.gellmann.all_gellmann_matrix(3, with_I=False)
for x in gm3:
    print(x.real if (x.imag.max() < 1e-4) else x)


## d=2 Pauli

Generalized Gell-Mann matrices are also introduced when $d$ is not $3$ [wiki/generalization-pauli](https://en.wikipedia.org/wiki/Generalizations_of_Pauli_matrices). And specifically, when $d=2$, the generalized Gell-Mann matrices are the Pauli matrices. For brief, we will call the generalized Gell-Mann matrices as Gell-Mann matrices. 

In [ ]:
gm2 = numqi.gellmann.all_gellmann_matrix(2, with_I=False)
for x in gm2:
    print(x.real if (x.imag.max() < 1e-4) else x)


## d=4

For $d$-dimensional Gell-Mann matrices, there is $d^2-1$ matrices. The first $d(d-1)/2$ matrices are real symmetric matrices (Pauli-X like), the following $d(d-1)/2$ matrics are pure imaginary anti-symmetric matrices (Pauli-Y like), and the last $d-1$ matrices are real diagonal matrices (Pauli-Z like). They are orthogonal to each other with respect to the Hilbert-Schmidt inner product and they make up a basis for the space of tracelss Hermitian matrices.

In [ ]:
d = 4
gmd = numqi.gellmann.all_gellmann_matrix(d, with_I=False)

print('Pauli-X like Gell-Mann matric, real, symmetric, off-diagonal')
ind0 = np_rng.integers(0, d*(d-1)//2)
print(f'$M_{ind0}$=\n', gmd[ind0].real)

print('Pauli-Y like Gell-Mann matric, pure imaginary, anti-symmetric, off-diagonal')
ind1 = np_rng.integers(d*(d-1)//2, d*(d-1))
print(f'$M_{ind1}$=\n', gmd[ind1])

print('Pauli-Z like Gell-Mann matric, real, diagonal')
ind2 = np_rng.integers(d*(d-1), d*d-1)
print(f'$M_{ind2}$=\n', gmd[ind2].real)


## identity matrix

Sometimes, the identity matrix is also included in the Gell-Mann matrices to make a complete, orthogonal basis for Hermitian basis. In this case, the number of Gell-Mann matrices is $d^2$. `numqi.gellmann` module append this identity matrix $M_{d^2-1}=\sqrt{2/d}I$ to the Gell-Mann matrices as the last element.

"Orthogonal" means that the Hilbert-Schmidt inner product of any two matrices is zero. The Hilbert-Schmidt inner product of two matrices $A$ and $B$ is defined as $\langle A, B \rangle = \mathrm{Tr}(A^\dagger B)$. As for the Gell-Mann matrices, their Hilbert-Schmidt inner product

$$ \langle M_i,M_j\rangle=2\delta_{ij} $$

where $\delta_{ij}$ is the Kronecker delta and the factor $2$ is due to the normalization of the Gell-Mann matrices (so that when $d=2$, it will recover the Pauli matrices).

In [ ]:
gmd = numqi.gellmann.all_gellmann_matrix(d) #with_I=True (default)
print(f'identity: \n{gmd[-1].real}')

mat0,mat1 = gmd[np_rng.permutation(d*d)[:2]] # randomly pick two matrix
print(f'mat0=\n{mat0}\nmat1=\n{mat1}')
print('<mat0,mat0>=', np.trace(mat0.conj().T @ mat0))
print('<mat1,mat1>=', np.trace(mat1.conj().T @ mat1))
print('<mat0,mat1>=', np.trace(mat0.conj().T @ mat1))


"Complete" basis means any Hermitian matrix can be expressed as a linear combination of the Gell-Mann matrices with real coefficients, and any matrix (Hermitian or non-Hermitian) can be expressed as a linear combination of the Gell-Mann matrices with complex coefficients.

$$ A=\sum_i a_i M_i,a_i=\langle M_i,A \rangle/2. $$

When the matrix $A$ is Hermitian, the coefficients $a_i$ are real. When the matrix $A$ is non-Hermitian, the coefficients $a_i$ are complex.

In [ ]:
d = 3

print('non-Hermitian matrix')
matA = np.around(np_rng.uniform(-1,1,size=(d,d)) + 1j*np_rng.uniform(-1,1,size=(d,d)), 3)
coeff = numqi.gellmann.matrix_to_gellmann_basis(matA)
print(f'matA=\n{matA}')
print(f'coeff=\n{coeff}')

print('\nHermitian matrix')
matA = matA + matA.T.conj()
coeff = numqi.gellmann.matrix_to_gellmann_basis(matA)
print(f'matA=\n{matA}')
print(f'coeff.real=\n{coeff.real}')
print(f'coeff.imag=\n{coeff.imag}')

matA_restored = numqi.gellmann.gellmann_basis_to_matrix(coeff)
print(f'matA_restored=\n{matA_restored}')


## connection with Frobenius inner product

Given two matrix $A,B\in\mathbb{C}^{m\times n}$, their Frobenius inner product [wiki](https://en.wikipedia.org/wiki/Frobenius_inner_product) is defined

$$ \langle A,B\rangle_{fro}=\sum_{ij} A_{ij}^* B_{ij}=\mathrm{Tr}[A^\dagger B] $$

If we choose the Gell-Mann matrices as the basis for the space of $n\times n$ matrices, then the Frobenius inner product of two matrices is related to the inner product of two coefficient vectors.

$$ A=\sum_i a_i M_i,a_i=\langle M_i,A \rangle/2. $$

$$ B=\sum_i b_i M_i,b_i=\langle M_i,B \rangle/2. $$

$$ \langle A,B\rangle_{fro} = 2\langle \vec{a},\vec{b}\rangle=2\sum_i{a_i^*b_i} $$


In [ ]:
d = 4
matA = np.around(np_rng.uniform(-1,1,size=(d,d)) + 1j*np_rng.uniform(-1,1,size=(d,d)), 3)
coeffA = numqi.gellmann.matrix_to_gellmann_basis(matA)
matB = np.around(np_rng.uniform(-1,1,size=(d,d)) + 1j*np_rng.uniform(-1,1,size=(d,d)), 3)
coeffB = numqi.gellmann.matrix_to_gellmann_basis(matB)

print('<A,B>=', np.trace(matA.T.conj() @ matB))
print('<coeffA,coeffB>=', 2*np.vdot(coeffA, coeffB))


## connection with density matrix

Vectorization of density matrix see paper "Detecting entanglement by pure bosonic extension" [arxiv-link](https://arxiv.org/abs/2209.10934)

The density matrix is a Hermitian matrix with trace $1$. The density matrix can be expanded as a linear combination of the Gell-Mann matrices with real coefficients. For a $d$-dimensional system,

$$ \rho=\rho_0+\sum_{i=0}^{d^2-2}\vec{\rho}_iM_i $$

where $\vec{\rho}$ is a real vector with $d^2-1$ components and $\rho_0$ is the maximally mixed state. Apparently, the maximally mixed state $\rho_0$ will correspond to the zero vector $\vec{\rho}=0$.

In [ ]:
rho0 = np.eye(3)/3
vec0 = numqi.gellmann.dm_to_gellmann_basis(rho0)
print(f'vec(maximally mixed state)=\n{vec0}')

rho = numqi.random.rand_density_matrix(3)
vec = numqi.gellmann.dm_to_gellmann_basis(rho)
print(f'vec(random density matrix)=\n{vec}')


The purity $\gamma_{\rho}$ of a density matrix $\rho$ is closely related to the norm of the coefficient vector $\vec{\rho}$

$$\gamma_{\rho}=\mathrm{Tr}\left[ \rho ^2 \right] =\frac{1}{d}+2\lVert\vec{\rho}\rVert^{2}_2$$

It's easy to see that the purity is always larger than $1/d$ and the maximum mixed state $\rho=I/d$ gives the minimum purity $1/d$.


In [ ]:
d = 4
rho = numqi.random.rand_density_matrix(d)
vec = numqi.gellmann.dm_to_gellmann_basis(rho)
purity = np.trace(rho @ rho).real
purity_from_vec = 1/d + 2*np.vdot(vec, vec).real
print(f'purity={purity}, purity_from_vec={purity_from_vec}')


TODO some conjectures about the length of the density matrix vector

$$ \min\left\{ \left\Vert \vec{\rho}\right\Vert_2 :\rho\succeq0,\mathrm{Tr}[\rho]=1,\mathrm{rank}(\rho)=1\right\} $$

$$ \max\left\{ \left\Vert \vec{\rho}\right\Vert_2 :\rho\succeq0,\mathrm{Tr}[\rho]=1,\mathrm{rank}(\rho)=1\right\} $$

$$ \min\left\{ \left\Vert \vec{\rho}\right\Vert_2 :\rho\succeq0,\mathrm{Tr}[\rho]=1,\mathrm{rank}(\rho)<d\right\} $$

$$ \max\left\{ \left\Vert \vec{\rho}\right\Vert_2 :\rho\succeq0,\mathrm{Tr}[\rho]=1,\mathrm{rank}(\rho)<d\right\} $$
